##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 início rápido para iniciantes

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Ver em TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Executar no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver código fonte no GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Baixar notebook</a>
  </td>
</table>

Esta breve introdução usa [Keras] (https://www.tensorflow.org/guide/keras/overview) para:

1. Criar uma rede neural que classifique imagens.
2. Treinar essa rede neural.
3. E, finalmente, avaliar a precisão do modelo.
4. Alteracoes para adicionar early stopper

Este é um arquivo de bloco de notas [Google Colaboratory] (https://colab.research.google.com/notebooks/welcome.ipynb). Os programas Python são executados diretamente no navegador - uma ótima maneira de aprender e usar o TensorFlow. Para seguir este tutorial, execute o bloco de anotações no Google Colab clicando no botão na parte superior desta página.

1. No Colab, conecte-se a um tempo de execução do Python: No canto superior direito da barra de menus, selecione *CONNECT*.
2. Execute todas as células de código do notebook: Selecione *Tempo de execução* > *Executar tudo*.

Baixe e instale o pacote TensorFlow 2. Importe o TensorFlow para o seu programa:

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

Carregue e prepare o [conjunto de dados MNIST] (http://yann.lecun.com/exdb/mnist/). Converta as amostras de números inteiros em números de ponto flutuante:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Crie o modelo `tf.keras.Sequential` empilhando camadas. Escolha uma função otimizadora e de perda para treinamento:

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
# This callback will stop the training when there is no improvement in
# the loss for three consecutive epochs.

Treine e avalie o modelo:

In [4]:
backup = tf.config.get_visible_devices("GPU")

In [5]:
anterior = tf.config.get_visible_devices()
print(anterior, backup)
# Hide GPU from visible devices
#tf.config.set_visible_devices([], 'GPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

tf.config.set_visible_devices([], 'GPU')

In [7]:
import time 
start = time.time()
# Não faz diferenca usar esse with, vai sempre pra GPU
with tf.device('/CPU:0'):
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    # This callback will stop the training when there is no improvement in
    # the loss for three consecutive epochs.
    
    model.fit(x_train, y_train, epochs=3, callbacks=[callback])

    model.evaluate(x_test,  y_test, verbose=2)
print(time.time() - start)

Epoch 1/3
1875/1875 [==============================] - 1s 750us/step - loss: 0.2967 - accuracy: 0.9126
Epoch 2/3
1875/1875 [==============================] - 1s 749us/step - loss: 0.1430 - accuracy: 0.9573
Epoch 3/3
1875/1875 [==============================] - 1s 750us/step - loss: 0.1058 - accuracy: 0.9682
313/313 - 0s - loss: 0.0904 - accuracy: 0.9731
5.3995044231414795


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [9]:
import time 
start = time.time()
with tf.device('/GPU:0'):
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    # This callback will stop the training when there is no improvement in
    # the loss for three consecutive epochs.
    
    model.fit(x_train, y_train, epochs=10, callbacks=[callback])

    model.evaluate(x_test,  y_test, verbose=2)
print(time.time() - start)

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2966 - accuracy: 0.9134
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1416 - accuracy: 0.9573
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1073 - accuracy: 0.9673
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0879 - accuracy: 0.9725
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0751 - accuracy: 0.9764
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0650 - accuracy: 0.9792
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0582 - accuracy: 0.9812
Epoch 8/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0524 - accuracy: 0.9826
Epoch 9/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0486 - accuracy: 0.9840
Epoch 10/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.044

O classificador de imagem agora é treinado para ~98% de acurácia neste conjunto de dados. Para saber mais, leia os [tutoriais do TensorFlow] (https://www.tensorflow.org/tutorials/).